### Gensim LDA

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Import nltk stopwords and spacy for lemmatization
from nltk.corpus import stopwords

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
import spacy
%matplotlib inline

# Enable logging for Gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


#### 1. Import data set

In [3]:
# Create a list of words
from os import listdir
from os.path import isfile, join

mypath = 'data/election'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
# Collect all Tweets available and store them as list of dicts
import json
all_tweets = []

for f in onlyfiles:
    full_name = mypath+'/'+f

    with open(full_name, 'r') as f:
        data = json.load(f)
        all_tweets.extend(data)

In [5]:
# Statistics
print('Number of keys assigned:', len(all_tweets[0].keys()), '\n')
print('Sample of a Tweet:', all_tweets[0]['text'], '\n')
print('Info available in a Tweet:', '\n', all_tweets[0].keys())
print('\n','Info avaialable in the whole set:', '\n', pd.DataFrame(all_tweets[:10]).columns.values)

Number of keys assigned: 28 

Sample of a Tweet: RT @Derksen_Gelul: Nederlands bekendste droogkloot, Marcel van Roosmalen sloopt D66 helemaal de moeder!! Luister zelf naar deze topper en g… 

Info available in a Tweet: 
 dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])

 Info avaialable in the whole set: 
 ['contributors' 'coordinates' 'created_at' 'display_text_range' 'entities'
 'extended_tweet' 'favorite_count' 'favorited' 'filter_level' 'geo' 'id'
 'id_str' 'in_reply_to_screen_name' 'in_reply_to_status_id'
 'in_reply_to_status_id_str' 'in_reply_to_user_id'
 'in_reply_to_user_id_str' 'is_quote_stat

In [6]:
# Extract the data (id and text), and put it into dict and list
text_dict = {} 
text_list = []
id_list = []
missing_tweets = 0

for tweet in all_tweets:
    if ('text' in tweet.keys()):
        if ('id' in tweet.keys()):
            my_id = tweet['id']
            if (tweet['text'] != None):
                text_dict[my_id] = tweet['text']
                id_list.append(my_id)
                text_list.append([tweet['text']])
                
            elif (tweet['text'] == None):
                missing_values = missing_tweets + 1

In [7]:
# Statistics
t = len(text_dict)
max_length = 0
average_length = 0

for tweet in text_list : 
    average_length = average_length + sum(len(i) for i in tweet) 
    if (sum(len(i) for i in tweet) > max_length) : 
        max_length = sum(len(i) for i in tweet)
        max_tweet = tweet

print('The numver of blank Tweets = ' + str(missing_tweets))
print('In total ' + str(t) + ' Tweets collected')
print('On average Tweet is ' + str(round(average_length / t)) + ' charachters long', '\n')
print('Longest Tweet is:', max_tweet, '\n')
print('It is', sum(len(i) for i in max_tweet), 'characters long')

The numver of blank Tweets = 0
In total 795867 Tweets collected
On average Tweet is 124 charachters long 

Longest Tweet is: ['@tacticsoftuchel Courtois&gt;Karius\nAzpi&gt;Clyne \nChristensen&lt;VVD (for now)\nZouma/Rudiger&gt;&gt;&gt;&gt;&gt;&gt;\nEmerson/Alonso&lt;Robertson… https://t.co/5YNNJog70t'] 

It is 170 characters long


In [8]:
# Let's inspect the Tweets
pd.DataFrame(text_list[:5])

,0
0,RT @Derksen_Gelul: Nederlands bekendste droogk...
1,"RT @GerBStruik: Niet alleen Ollongren, maar ni..."
2,"@JoostNiemoller Joost, trap er toch niet in. H..."
3,@EenVandaag @thierrybaudet @D66 @APechtold Ald...
4,Dolhuysbrug moet na tien jaar definitief van t...


#### 2. Remove emails and newline characters 

In [9]:
# Before
pprint(text_list[:3])

[['RT @Derksen_Gelul: Nederlands bekendste droogkloot, Marcel van Roosmalen '
  'sloopt D66 helemaal de moeder!! Luister zelf naar deze topper en g…'],
 ['RT @GerBStruik: Niet alleen Ollongren, maar niemand in #Rutte3 heeft '
  'kennelijk enig benul van staatsrecht. Een minister die als minister ee…'],
 ['@JoostNiemoller Joost, trap er toch niet in. Hier is één grote minne '
  'manipulatie aan de gang tegen FVD. Deze heeft… https://t.co/lVFOuFFYF3']]


In [10]:
# data = df.content.values.tolist()
# TODO: include other Tweets into analysis
data = text_list[:10000]

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', str(sent)) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", str(sent)) for sent in data]

# TODO: remove https and links
data = [re.sub("https", "", str(sent)) for sent in data]

In [11]:
# After
pprint(data[:3])

['[RT Nederlands bekendste droogkloot, Marcel van Roosmalen sloopt D66 '
 'helemaal de moeder!! Luister zelf naar deze topper en g…]',
 '[RT Niet alleen Ollongren, maar niemand in #Rutte3 heeft kennelijk enig '
 'benul van staatsrecht. Een minister die als minister ee…]',
 'Joost, trap er toch niet in. Hier is één grote minne manipulatie aan de gang '
 'tegen FVD. Deze heeft… ://t.co/lVFOuFFYF3]']


#### 3. Tokenize words and Clean-up text

In [12]:
def sent_to_words(sentences) :
    '''Split sentences into words'''
    for sentence in sentences :
        # The main function here is Gensim's simple_preprocess
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

# Let's explore what do we have now
pd.DataFrame(data_words[:5])

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,rt,nederlands,bekendste,droogkloot,marcel,van,roosmalen,sloopt,helemaal,de,...,luister,zelf,naar,deze,topper,en,None,None,None,None
1,rt,niet,alleen,ollongren,maar,niemand,in,rutte,heeft,kennelijk,...,benul,van,staatsrecht,een,minister,die,als,minister,ee,None
2,joost,trap,er,toch,niet,in,hier,is,een,grote,...,manipulatie,aan,de,gang,tegen,fvd,deze,heeft,co,lvfouffyf
3,aldus,leugenaar,hoe,durft,deze,partij,nog,de,van,co,...,kqfumv,None,None,None,None,None,None,None,None,None
4,dolhuysbrug,moet,na,tien,jaar,definitief,van,tafel,vvd,haarlem,...,saanwvxg,nmaak,uw,eigen,filter,ove,co,pbveexkhtf,None,None


#### 4. Remove Stopwords, Make Bigrams and Lemmatize

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

In [13]:
# NLTK stop words collection
stop_words = stopwords.words('dutch')

# TODO: Extend stop words collection using native speaker knowledge
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [14]:
print('The number of Dutch stop words avaialable:', len(stop_words))

The number of Dutch stop words avaialable: 101


In [15]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[5]]])

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['rt', 'niet', 'alleen_ollongren', 'maar', 'niemand', 'in', 'rutte', 'heeft', 'kennelijk_enig_benul', 'van', 'staatsrecht', 'een', 'minister', 'die', 'als', 'minister_ee']


In [16]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts) :
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts) :
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts) :
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']) :
    texts_out = []
    for sent in texts :
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Lemmatization is converting a word to its root word. Spacy NLP package responsible for that. Thanks we have Dutch language version.

In [17]:
# Remove stop words
data_words_nostops = remove_stopwords(data_words)

# Form bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

nlp = spacy.load('nl', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['nederlands_bekendste', 'droogkloot_marcel', 'roosmalen_sloopt', 'helemaal', 'moeder_luister', 'topper']]


In [18]:
print(data_words[:1])

[['rt', 'nederlands', 'bekendste', 'droogkloot', 'marcel', 'van', 'roosmalen', 'sloopt', 'helemaal', 'de', 'moeder', 'luister', 'zelf', 'naar', 'deze', 'topper', 'en']]


In [19]:
print(data_words_nostops[:1])

[['rt', 'nederlands', 'bekendste', 'droogkloot', 'marcel', 'roosmalen', 'sloopt', 'helemaal', 'moeder', 'luister', 'topper']]


In [20]:
print(data_words_bigrams[:1])

[['rt', 'nederlands_bekendste', 'droogkloot_marcel', 'roosmalen_sloopt', 'helemaal', 'moeder_luister', 'topper']]


In [21]:
print(data_lemmatized[:1])

[['nederlands_bekendste', 'droogkloot_marcel', 'roosmalen_sloopt', 'helemaal', 'moeder_luister', 'topper']]


In [3]:

import time
import pickle
import bs4

# Load results
with open('elections_data_lemmatized', 'rb') as fp:
    data_lemmatized = pickle.load(fp)

#### 6. Create the Dictionary and Corpus needed for Topic Modeling

In [4]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Corpus contains pairs - a unique id for each word in the document and its frequency (word_id, word_frequency).
print(pd.DataFrame(corpus[:1]))

        0       1       2       3       4       5
0  (0, 1)  (1, 1)  (2, 1)  (3, 1)  (4, 1)  (5, 1)


In [5]:
# We also can check what was the word from the dictionary - id2word
pprint(corpus[:2])
print(id2word[11])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1)]]
ollongren


In [6]:
# Readable format
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('droogkloot_marcel', 1),
  ('helemaal', 1),
  ('moeder_luister', 1),
  ('nederlands_bekendste', 1),
  ('roosmalen_sloopt', 1),
  ('topper', 1)],
 [('alleen', 1),
  ('benul', 1),
  ('ee', 1),
  ('kennelijk_enig', 1),
  ('minister', 2),
  ('ollongren', 1),
  ('rutte', 1),
  ('staatsrecht', 1)]]

#### 7. Build the model

In [ ]:
# TODO: tune the parameters
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, # corpus
                                           id2word=id2word, # dict
                                           num_topics=10, # number of topic to be extracted
                                           random_state=100,
                                           update_every=1, #
                                           chunksize=100, # 
                                           passes=10, # 
                                           alpha='auto', # 
                                           per_word_topics=True) # 

#### 8. Print the topics

In [ ]:
# Print the keywords in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

#### 9. Compute Model Perplexity and Coherence Score

In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.29433185874

Coherence Score:  0.520018195805


#### 10. Visualize the topics-keywords

Each bubble is a topic. The larger the bubble, the more prevalent the topic.

In [28]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      14.208471        1       1  0.312178 -0.003845
6      12.012739        1       2  0.204223 -0.019000
0      10.645026        1       3 -0.089848  0.380279
7      10.387444        1       4 -0.110486 -0.135900
9      10.173903        1       5 -0.047922 -0.064508
2       8.977041        1       6  0.088315  0.043885
8       8.544526        1       7 -0.102245 -0.045733
4       8.480484        1       8 -0.099838 -0.037474
1       8.457304        1       9 -0.066007 -0.069451
5       8.113062        1      10 -0.088369 -0.048253, topic_info=     Category         Freq                    Term        Total  loglift  \
term                                                                       
83    Default   965.000000                 racisme   965.000000  30.0000   
49    Default   798.000000                  baudet   798.000000  29.0000   
20    Default  1434.000000                      co  1434.000000  28.0000   
339   Default   669.000000                  gewoon   669.000000  27.0000   
674   Default   833.000000              referendum   833.000000  26.0000   
648   Default   702.000000                     mee   702.000000  25.0000   
476   Default   628.000000                pechtold   628.000000  24.0000   
241   Default   621.000000              uitspraken   621.000000  23.0000   
323   Default   590.000000                   media   590.000000  22.0000   
36    Default   670.000000                gemeente   670.000000  21.0000   
51    Default   568.000000                     fvd   568.000000  20.0000   
138   Default   554.000000                    doet   554.000000  19.0000   
60    Default   573.000000                     mag   573.000000  18.0000   
182   Default   607.000000                    weer   607.000000  17.0000   
65    Default   485.000000                    keer   485.000000  16.0000   
346   Default   440.000000                    pvda   440.000000  15.0000   
86    Default   547.000000                    gaat   547.000000  14.0000   
1758  Default   498.000000                      nl   498.000000  13.0000   
756   Default   497.000000                      af   497.000000  12.0000   
11    Default   418.000000                   rutte   418.000000  11.0000   
1756  Default   477.000000                   hallo   477.000000  10.0000   
1755  Default   475.000000       detailhandel_werk   475.000000   9.0000   
1779  Default   475.000000                    veil   475.000000   8.0000   
288   Default   396.000000                leefbaar   396.000000   7.0000   
184   Default   379.000000                donorwet   379.000000   6.0000   
75    Default   433.000000                 kabinet   433.000000   5.0000   
69    Default   385.000000                    vind   385.000000   4.0000   
219   Default   353.000000           discriminatie   353.000000   3.0000   
70    Default   375.000000               zorgelijk   375.000000   2.0000   
6147  Default   416.000000                schaffen   416.000000   1.0000   
...       ...          ...                     ...          ...      ...   
7704  Topic10   112.566224               weg_terug   113.434647   2.5040   
7702  Topic10   112.566224  nederland_afgeschreven   113.434647   2.5040   
945   Topic10    91.426533               interview    92.253763   2.5027   
478   Topic10    85.806876              debatteren    86.635371   2.5021   
213   Topic10    68.866464                     net    69.693759   2.4998   
1274  Topic10    60.898011                    kijk    61.725429   2.4982   
8956  Topic10    66.490697                verbeter    67.425146   2.4977   
8955  Topic10    66.490697        halfbakken_motie    67.425146   2.4977   
8954  Topic10    66.490697           buhne_deining    67.425146   2.4977   
348   Topic10    48.027637                     zie    48.854733   2.4946   
151   Topic10    45.351066               

### LDA Mallet Model